In [1]:
import cv2
import numpy as np
import pandas as pd
import os
from skimage.feature import hog, local_binary_pattern
from sklearn.decomposition import PCA
from concurrent.futures import ProcessPoolExecutor
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from concurrent.futures import ThreadPoolExecutor
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
labels_df = pd.read_csv('label.csv')

image_directory = 'data'

In [3]:
# Function to extract ORB features
def extract_orb_features(image, max_features=128):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    if descriptors is not None:
        if descriptors.shape[0] > max_features:
            descriptors = descriptors[:max_features, :]
        elif descriptors.shape[0] < max_features:
            padding = np.zeros((max_features - descriptors.shape[0], descriptors.shape[1]))
            descriptors = np.vstack((descriptors, padding))
        return descriptors.flatten()
    else:
        return np.zeros(max_features * 32)

# Function to extract HOG features
def extract_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = hog(gray_image, pixels_per_cell=(24, 24), block_norm='L2-Hys')
    return features

# Function to extract LBP features
def extract_lbp_features(image, radii=[1, 2, 3], n_points=8, method='uniform'):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp_features = []
    
    for radius in radii:
        lbp = local_binary_pattern(gray_image, n_points, radius, method=method)
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        lbp_features.extend(hist)
    
    return np.array(lbp_features) 

# Function to extract color histogram features
def extract_color_histogram(image):
    # Load the image
    
    # Compute the histogram for each color channel (B, G, R)
    hist_b = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
    hist_r = cv2.calcHist([image], [2], None, [256], [0, 256])
    
    # Normalize the histograms
    hist_b = cv2.normalize(hist_b, hist_b).flatten()
    hist_g = cv2.normalize(hist_g, hist_g).flatten()
    hist_r = cv2.normalize(hist_r, hist_r).flatten()

    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate([hist_b, hist_g, hist_r])

    # print(hist_features.shape)

    return hist_features

# Function to extract SIFT features
def extract_sift_features(image, max_features=10):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is not None:
        if descriptors.shape[0] > max_features:
            descriptors = descriptors[:max_features, :]
        elif descriptors.shape[0] < max_features:
            padding = np.zeros((max_features - descriptors.shape[0], descriptors.shape[1]))
            descriptors = np.vstack((descriptors, padding))
        # print("yeyyy")
        return descriptors.flatten()
    else:
        return np.zeros(max_features * 128)

# Combine all features into a single feature vector
def extract_combined_features(image):
    # Extract various features
    orb_features = extract_orb_features(image)
    hog_features = extract_hog_features(image)
    lbp_features = extract_lbp_features(image)
    color_histogram = extract_color_histogram(image)
    sift_features = extract_sift_features(image)

    # Combine all features into a single array
    combined_features = np.concatenate((
        orb_features, 
        hog_features, 
        lbp_features, 
        color_histogram, 
        sift_features, 
        # gftt_features, 
        # sobel_features
    ))
    return combined_features


In [4]:
X = labels_df['filename']
y = labels_df['label']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [6]:
def process_image(image_name, label):
    # print(image_name, label)
    image_path = os.path.join(image_directory, image_name)
    try:
        image = cv2.imread(image_path)
    except:
        return None

    if image is None:
        return None

    # Resize image
    resized_image = cv2.resize(image, (250, 200))

    # Extract combined features
    combined_features = extract_combined_features(resized_image)
    # print(len(combined_features))

    # Return the features with the label
    return np.append(combined_features, label)


In [7]:

def extract_features(image_names, labels):
    results = []

    def process_and_store(image_name, label):
        result = process_image(image_name, label)
        if result is not None:
            results.append(result)

    with ThreadPoolExecutor() as executor:
        # Submit tasks to the executor
        executor.map(process_and_store, image_names, labels)

    return np.array(results)

In [8]:
print(X_train.head())

12359    Image_12360.jpg
9243      Image_9244.jpg
11838    Image_11839.jpg
3524      Image_3525.jpg
3475      Image_3476.jpg
Name: filename, dtype: object


In [9]:
print(y_train.head())

12359     cycling
9243     drinking
11838    clapping
3524      texting
3475      cycling
Name: label, dtype: object


In [10]:
print("Shape of train_features:", X_train.shape)
print("Shape of test_features:", X_test.shape)

Shape of train_features: (10080,)
Shape of test_features: (2520,)


In [11]:
train_features = extract_features(X_train, y_train)
test_features = extract_features(X_test, y_test)

# Separate features and labels
X_train_features = train_features[:, :-1]
y_train_labels = train_features[:, -1]

X_test_features = test_features[:, :-1]
y_test_labels = test_features[:, -1]





[ WARN:0@0.896] global loadsave.cpp:241 findDecoder imread_('data/Image_9244.jpg'): can't open/read file: check file path/integrity
[ WARN:9@0.901] global loadsave.cpp:241 findDecoder imread_('data/Image_7832.jpg'): can't open/read file: check file path/integrity
[ WARN:11@0.902] global loadsave.cpp:241 findDecoder imread_('data/Image_8036.jpg'): can't open/read file: check file path/integrity
[ WARN:12@0.902] global loadsave.cpp:241 findDecoder imread_('data/Image_977.jpg'): can't open/read file: check file path/integrity
[ WARN:12@0.903] global loadsave.cpp:241 findDecoder imread_('data/Image_9751.jpg'): can't open/read file: check file path/integrity
[ WARN:16@0.904] global loadsave.cpp:241 findDecoder imread_('data/Image_8218.jpg'): can't open/read file: check file path/integrity
[ WARN:19@0.904] global loadsave.cpp:241 findDecoder imread_('data/Image_9319.jpg'): can't open/read file: check file path/integrity
[ WARN:19@0.905] global loadsave.cpp:241 findDecoder imread_('data/Image

In [12]:
print(X_train.shape)
print(X_test.shape)
print(train_features.shape)
print(test_features.shape)


(10080,)
(2520,)
(8063, 10063)
(2009, 10063)


In [13]:
# Apply PCA only on the training data to avoid data leakage
pca = PCA(n_components=250)  # Adjust the number of components as needed

# Normalize features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)
X_test_scaled = scaler.transform(X_test_features)

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)



In [14]:
best_rf_params = {
    "n_estimators": 1000,
    "min_samples_split": 10,
    "max_features":'sqrt',
    "max_depth":30,
    "random_state": 42,
    "n_jobs": -1

}

rf = RandomForestClassifier(**best_rf_params)
rf.fit(X_train_pca, y_train_labels)

# Test the model
y_pred = rf.predict(X_test_pca)
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Random Forest Model Accuracy: {accuracy:.4f}")

Random Forest Model Accuracy: 0.3225


In [15]:
model_filename = 'trained_random_forest_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(rf, file)

In [16]:
# Create a pipeline with imputation and Naive Bayes
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('nb', GaussianNB())
])

# Train the model
pipeline.fit(X_train_pca, y_train_labels)

# Test the model
y_pred = pipeline.predict(X_test_pca)

# Calculate the accuracy
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Naive Bayes Model Accuracy: {accuracy:.4f}")

Naive Bayes Model Accuracy: 0.2708


In [17]:
# Create a pipeline with imputation and Perceptron
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('perceptron', Perceptron(max_iter=1000, tol=1e-3, random_state=42))
])

# Train the model
pipeline.fit(X_train_pca, y_train_labels)

# Test the model
y_pred = pipeline.predict(X_test_pca)

# Calculate the accuracy
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Perceptron Model Accuracy: {accuracy:.4f}")

Perceptron Model Accuracy: 0.2046


In [18]:
# Create a pipeline with imputation and MLPClassifier
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('mlp', MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42))
])

# Train the model
pipeline.fit(X_train_pca, y_train_labels)

# Test the model
y_pred = pipeline.predict(X_test_pca)

# Calculate the accuracy
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Multilayer Perceptron Model Accuracy: {accuracy:.4f}")

Multilayer Perceptron Model Accuracy: 0.2778


In [19]:
# Create individual classifiers
nb = GaussianNB()
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
perceptron = Perceptron(max_iter=1000, tol=1e-3, random_state=42)

# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('naive_bayes', nb),
    ('decision_tree', dt),
    ('random_forest', rf),
    ('perceptron', perceptron)
], voting='hard')  # Use 'soft' for probabilistic voting

# Create a pipeline with imputation and the ensemble classifier
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('voting_clf', voting_clf)
])

# Train the model
pipeline.fit(X_train_pca, y_train_labels)

# Test the model
y_pred = pipeline.predict(X_test_pca)

# Calculate the accuracy
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Ensemble Model Accuracy: {accuracy:.4f}")

Ensemble Model Accuracy: 0.3021


In [20]:
# # model_filename = 'trained_random_forest_model.pkl'
# # with open(model_filename, 'rb') as file:
# #     loaded_model = pickle.load(file)

# # importances = loaded_model.feature_importances_

# importances = rf.feature_importances_

# # Define feature sizes
# edges_size = 100
# orb_size = 0
# hog_size = 3888
# lbp_size = 30
# color_histogram_size = 768  # Assuming 256 bins for each of B, G, R channels

# # Sizes for new features
# harris_size = 100  # Assuming PCA reduced Harris features to 100
# gabor_size = 100   # Assuming PCA reduced Gabor features to 100
# sobel_size = 100   # Assuming PCA reduced Sobel features to 100
# gftt_size = 200    # Assuming PCA reduced GFTT features to 100
# glcm_size = 256 

# # Split importances back into the respective feature categories
# edges_importance = importances[:edges_size]
# orb_importance = importances[edges_size:edges_size + orb_size]
# hog_importance = importances[edges_size + orb_size:edges_size + orb_size + hog_size]
# lbp_importance = importances[edges_size + orb_size + hog_size:edges_size + orb_size + hog_size + lbp_size]
# color_histogram_importance = importances[edges_size + orb_size + hog_size + lbp_size:edges_size + orb_size + hog_size + lbp_size + color_histogram_size]

# # Adding Harris, Gabor, Sobel, and GFTT features
# start_idx = edges_size + orb_size + hog_size + lbp_size + color_histogram_size
# harris_importance = importances[start_idx:start_idx + harris_size]
# gabor_importance = importances[start_idx + harris_size:start_idx + harris_size + gabor_size]
# sobel_importance = importances[start_idx + harris_size + gabor_size:start_idx + harris_size + gabor_size + sobel_size]
# gftt_importance = importances[start_idx + harris_size + gabor_size + sobel_size:start_idx + harris_size + gabor_size + sobel_size + gftt_size]
# glcm_importance = importances[start_idx + harris_size + gabor_size + sobel_size + gftt_size:start_idx + harris_size + gabor_size + sobel_size + gftt_size + glcm_size]

# # Sum the importances for each category
# category_importances = {
#     'edges': (np.sum(edges_importance), edges_size),
#     'orb_features': (np.sum(orb_importance), orb_size),
#     'hog_features': (np.sum(hog_importance), hog_size),
#     'lbp_features': (np.sum(lbp_importance), lbp_size),
#     'color_histogram': (np.sum(color_histogram_importance), color_histogram_size),
#     'harris_features': (np.sum(harris_importance), harris_size),
#     'gabor_features': (np.sum(gabor_importance), gabor_size),
#     'sobel_features': (np.sum(sobel_importance), sobel_size),
#     'gftt_features': (np.sum(gftt_importance), gftt_size),
#     'glcm_features': (np.sum(glcm_importance), glcm_size)
# }

# # Convert to pandas DataFrame for easier viewing
# category_importances_df = pd.DataFrame(category_importances, index=['Importance', 'Feature Count']).T

# # Print the importances by category and feature count
# print(category_importances_df.sort_values(by='Importance', ascending=False))

In [21]:
# # Compute the confusion matrix
# cm = confusion_matrix(y_test, y_pred)

# # Plot the confusion matrix using seaborn
# plt.figure(figsize=(10, 7))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=set(y), yticklabels=set(y))
# plt.xlabel("Predicted")
# plt.ylabel("Actual")
# plt.title("Confusion Matrix")
# plt.show()